# Spotify Audio Analysis
Explanation of Variables [here](https://developer.spotify.com/documentation/web-api/reference/tracks/get-several-audio-features/)

In [1]:
dat = read.csv('nothing_faith.csv')
test_dat = read.csv('labeled_data.csv')
attach(dat)

In [3]:
typeof(dat)
dim(dat)

[1] "list"

[1] 200  20

In [ ]:
summary(dat)
summary(test_dat)

In [ ]:
# summary(dat)
# dat = dat[,-1]
# names(dat)
dat[101:105,]
dat[1:5,]
# test_dat[195:200,]
# dat[101:105,]

In [ ]:
# I left the uri in there so you can copy and paste it into the Spotify search bar and listen to
# some funky fresh tunes :)
dat[1:5,1:8]
dat[1:5,9:15]

In [ ]:
pairs(dat)

In [ ]:
length(energy)

In [ ]:
#looks like loudness and energy positively correlate!
plot(energy[1:100], loudness[1:100], col='red', xlab="energy", ylab="loudness") #good
points(energy[101:200], loudness[101:200], col='blue') #bad
# points(energy[201:300], loudness[101:200], col='green') #ok

Let's just work with binary classes for now (good and bad):

In [ ]:
new_dat = dat[1:200,]
okay_dat = dat[201:300,]

In [ ]:
names(dat)

In [ ]:
logModelA = glm(class~danceability,
                data=test_dat, family='binomial')

In [ ]:
logModelA

In [ ]:
logModel.probs = predict(logModelA, test_dat, type='response')
logModel.preds=rep('bad', dim(okay_dat)[1])

logModel.preds[logModel.probs>.75]='good'
logModel.preds[logModel.probs<.75]='okay'
logModel.preds[logModel.probs<.25]='bad'

In [ ]:
logModel.preds

In [ ]:
test_dat$class = logModel.preds

In [ ]:
#wasn't sure how to only print out the URI and class..
test_dat[1:5,]

# THIS IS ACTUALLY SO EXCITING WOW IF I WEREN'T IN PUBLIC I'D PROBABLY BE SCREAMING OR SOMETHING

The goal of this project is to be able to input a Spotify URI and then classify the song as ```good```, ```okay```, or ```bad``` based on it's audio features. I want to make this super-app-y at a Hackathon that I'm going to this weekend (sorry going to miss class on Monday :( ) but THIS IS REALLY EXCITING.

There were some Spotify API logic things that I need to handle--one call to recieve song features has a limit of 100 songs, so I need to find ways to handle that--which is why there's only 100 of each category right now. I wanted to throw something together really quick but I'll be doing more in depth analysis later on.

I didn't actually train my model on my 'okay' classes, but instead used that as my 'testing set'. The way I classified my songs:

I generally get tired of playing the same songs over and over again, so I usually create Spotify playlists by date (i.e: ```1/31/18 - 3/25/18```, ```3/25/18 - 5/25/18```, ...)  

__Good__ songs are songs that were able to make it through at least two playlists. So if I didn't get tired of a song by the time I created my next time period playlist, I classified that as a good song.

__Okay__ songs did not make it through to a next time period. I got tired of the song by the end of the ~2 months.

__Bad__ songs are mostly just the hit country songs that I found on a random playlist on Spoot (tbh I didn't even listen to it). Also EDM.

Going in and plugging in the songs that were classified as bad, I found that this those songs were pretty outlier-ish. Fitz and the Tantrum does lean towards dance-y electronic-y music, so it makes sense that it was classified as bad. Songs that were classified as good, I thought to myself, 'Oh! I did really like this song a while ago' or something like that. 

I am concerned that this project is a little too opinion-based and I that I will have really bad confirmation bias--maybe this model is utter trash, but I keep telling myself that this is really awesome, but IT IS.

In [ ]:
logModel.probs = predict(logModelA, dat, type='response')
logModel.preds=rep('good', dim(dat)[1])

# logModel.preds[logModel.probs<.75]='good'
logModel.preds[logModel.probs<.75]='okay'
logModel.preds[logModel.probs<.25]='bad'

In [ ]:
dat['prediction'] = logModel.preds
dat['prob'] = logModel.probs

In [ ]:
summary(logModel.probs[1:100])
summary(logModel.probs[101:200])
summary(logModel.probs[201:300])
match(max(logModel.probs[101:200]), logModel.probs)

In [ ]:
dat[50:60, 12:17]
dat[150:160, 12:17]
dat[250:260, 12:17]

In [ ]:
logModelA = 5
logModelA

In [ ]:
model= load("/home/mighty/gmc/gmc-backend/models/nothing_faith.rda")

In [ ]:
logModelA

## Trying to connect to PostgreSQL Server

In [ ]:
install.packages("RPostgreSQL")

In [ ]:
require(RPostgreSQL)

In [ ]:
drv <- dbDriver("PostgreSQL")

In [ ]:
con <- dbConnect(drv, dbname = 'd8hpli4tp47ijj',
                host="ec2-107-20-183-142.compute-1.amazonaws.com",
                port=5432,
                user="bzhnjcntyqxppx",
                password="c2910c0a80c3702108e24a12c11c1a6f7ab0a9ddfbd0ed7d63cb7b7707f32938")

In [ ]:
dbExistsTable(con, "test_msgs")

In [ ]:
myTable <- dbReadTable(con,c("test_msgs"))

In [ ]:
myTable

In [ ]:
res <- dbSendQuery(con, statement=paste("
INSERT INTO test_msgs VALUES ('Yo.', 'hello from R')")); 

In [ ]:
name = "Poo"
msg = "msgesgsd"

In [ ]:
res <- dbSendQuery(con, statement=paste("
INSERT INTO test_msgs VALUES ('",name,"','",msg,"')")); 

### Blobs/Bytea

In [ ]:
model

In [ ]:
x <- serialize(logModelA, NULL, ascii=TRUE)

In [ ]:
person = "Mighty"
person

In [ ]:
query=sprintf("SELECT * 
        FROM test_msgs 
        WHERE person = '%s'", person)

dbClearResult(res)
res <- dbSendQuery(con, statement=query);
dbFetch(res)

In [ ]:
x <- serialize(list(1,2,3), NULL)

In [ ]:
x_conv <- as.character(x)
x_conv
x_collapse = paste(x_conv, collapse=",")
x_collapse = paste("{", x_collapse,"}", sep="")
x_collapse

In [ ]:
# query = sprintf("INSERT INTO test_bytea VALUES (12, decode('%s', 'hex'))", x_conv)
query = sprintf("INSERT INTO test_bytea VALUES (2, '%s')", x_collapse)
res <- dbSendQuery(con, statement=query); 

In [ ]:
# query=sprintf("SELECT encode(model, 'hex') 
#         FROM test_bytea
#         WHERE id=12")
query = sprintf("SELECT * FROM test_bytea WHERE id=2")

dbClearResult(res)
res <- dbSendQuery(con, statement=query);
z = dbFetch(res)
z

In [ ]:
y = strsplit(z$model, ",")[[1]]
y[1] = substring(y[1], 2)
y[length(y)] = substring(y[length(y)], 1, nchar(y[length(y)]) - 1)
y

In [ ]:
new = as.raw(as.hexmode(y))
unserialize(new)

## Final

Serialize the logistic model into raw bits.

In [ ]:
x <- serialize(logModelA, NULL)

Save the bits as a string (? this is inefficient..)

In [ ]:
x_conv <- as.character(x)
x_collapse = paste(x_conv, collapse=",")
x_collapse = paste("{", x_collapse,"}", sep="")

In [ ]:
query = sprintf("INSERT INTO test_bytea VALUES (3,'%s')", x_collapse)
res <- dbSendQuery(con, statement=query); 

In [ ]:
query = sprintf("SELECT * FROM test_bytea WHERE id=3")

dbClearResult(res)
res <- dbSendQuery(con, statement=query);
z = dbFetch(res)
z

In [ ]:
y = strsplit(z$model, ",")[[1]]
y[1] = substring(y[1], 2)
y[length(y)] = substring(y[length(y)], 1, nchar(y[length(y)]) - 1)

In [ ]:
new = as.raw(as.hexmode(y))
unserialize(new)